In [1]:
## 2022 through 2023 analysis

import pandas as pd
import numpy as np
import regex as re
from pandarallel import pandarallel
import os
# from pathlib import Path
# from natsort import natsorted, ns
import glob
import pydash as dash
from polyfuzz import PolyFuzz
import rapidfuzz as fuzz
from rapidfuzz.process import extractOne
from rapidfuzz.distance import Levenshtein
from rapidfuzz.fuzz import ratio
from rapidfuzz import utils as fuzz_utils
# import itertools as it
# import json
# import spacy
# from polyfuzz import PolyFuzz
# from polyfuzz.models import EditDistance, TFIDF, RapidFuzz #Embeddings, SpacyEmbeddings
# from flair.embeddings import TransformerWordEmbeddings
# import tfidf_matcher as matcher
from ftfy import fix_text
# from textblob import TextBlob, Word, Blobber
# from collections import namedtuple, defaultdict, deque

pandarallel.initialize(progress_bar=False, verbose=1)
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)
# pd.set_option('display.max_rows', 100)
pd.set_option('display.min_rows', 30)
pd.set_option('display.max_colwidth', 100)
# pd.set_option('display.float_format', lambda x: '%.0f' % x)
from mlxtend.frequent_patterns import apriori, association_rules


In [2]:
cols = ['id', 'sku', 'order_lines', 'order_qty']

def join_df(args):
    df_list = []
    for arg in args:
        name = arg.split('_')[-1].replace('.xlsx', '')
        df = pd.read_excel(arg, usecols=cols)
        df['mk'] = name
        df_list.append(df)
    return df_list

files = glob.glob('eb-sh-az files/*.xlsx')
df = pd.concat(join_df(files), axis=0, ignore_index=True)

ddf = df.dropna(subset='order_lines', axis=0).reset_index(drop=True).copy()

def cleaner(x):
    to_remove = ['everest parts supplies ', 'everest parts brand ', 'everest brand ']
    vals = re.sub(r'|'.join(to_remove), '', x, flags=re.I)
    vals = vals.strip()
    if x.startswith('New '):
        vals = re.sub(r'^New ', '', x, flags=re.I)
        vals = vals.strip()
    x = vals
    return x

ddf['order_lines'] = ddf['order_lines'].apply(cleaner)

def analyze(*args, **kwargs):
    """
    analyze creates basket analysis with support and lift metrics
    Arguments:
    [
        args[0] = DataFrame,
        args[1] = Invoice or Order Id,
        args[2] = Barcode, or Comparible Basket Item
        args[3] = Quantity sold of each item within each order
    ]
    """

    def encodeUnits(x):
        if x <= 0:
            return False
        if x >= 1:
            return True

    def format_data(*args):
        cols = args[0].columns.to_list()
        df = args[0].copy()
        df[cols[3]] = df[cols[3]].astype(float)
        df = (args[0].groupby([cols[0], cols[1]])[cols[3]].sum().unstack().reset_index().fillna(0).set_index(cols[0]))
        df_enc = df.parallel_applymap(encodeUnits)
        df_enc_plus = df_enc[(df_enc > 0).sum(axis=1) >= 2]
        return df_enc_plus

    def apply_apriori(df, **kwargs):
        bk = apriori(df, min_support=kwargs.get('min_support', 0.002), use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)

        assoc = association_rules(bk, metric='lift', min_threshold=1).sort_values(by='lift', ascending=False).reset_index(drop=True)
        if kwargs.get('apply_join') == True:
            assoc[['antecedents', 'consequents']] = assoc[['antecedents', 'consequents']].applymap(lambda x: ', '.join(x))

        return assoc

    basket = format_data(*args)
    bk_df = apply_apriori(basket)
    return bk_df

df_bk = analyze(ddf, 'id', 'sku', 'order_qty')

df = df_bk.copy()

df.insert(0, 'sku1', np.nan)
df.insert(2, 'sku2', np.nan)

# df['sku'] = np.where(df['antecedents'].isin(ddf['order_lines']) == True, df['antecedents'].map(ddf.set_index('order_lines')['sku'].to_dict()), df['consequents'].map(ddf.set_index('order_lines')['sku'].to_dict()))

df['sku1'] = df['antecedents'].map(ddf.set_index('order_lines')['sku'].to_dict())

mapper = ddf.set_index('order_lines')['sku'].to_dict()
def skus(x, vals):
    matches = []
    for i in x:
        if vals.get(i) is not None:
            matches.append(vals[i])
    return matches

df['sku1'] = df['antecedents'].apply(lambda x: skus(x,mapper))
df['sku2'] = df['consequents'].apply(lambda x: skus(x,mapper))
# df[['antecendents', 'consequents_']] = df[['antecedents', 'consequents']].applymap(lambda x: '||'.join(x))
df[['antecedents', 'consequents']] = df[['antecedents', 'consequents']].applymap(lambda x: list(x))

# df['sku2'] = df['consequents'].map(ddf.set_index('order_lines')['sku'].to_dict())

df.to_excel('mk_analysis_22-23_V2.xlsx', index=False)

# df_bk.to_excel('mk_analysis_22-23.xlsx', index=False)
